In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/langchain-google-memorystore-redis-python/blob/main/samples/langchain_quick_start.ipynb)

---

# **Introduction**
In this codelab, you'll learn how to create a powerful interactive generative AI application using Retrieval Augmented Generation powered by [Memorystore for Redis](https://cloud.google.com/memorystore) and [LangChain](https://www.langchain.com/). You'll build an application grounded in a [Netflix Movie dataset](https://www.kaggle.com/datasets/shivamb/netflix-shows), allowing you to interact with movie data in exciting new ways.

## Prerequisites

* A basic understanding of the Google Cloud Console
* Basic skills in command line interface and Google Cloud shell
* Basic python knowledge

## What You Will Learn

* How to deploy a Memorystore for Redis instance
* How to use Memorystore for Redis as a VectorStore
* How to use Memorystore for Redis as a DocumentLoader
* How to use Memorystore for Redis as ChatHistory storage

# **Setup and Requirements**

In the following instructions you will learn to:
1. Install required dependencies for your application
2. Set up authentication for your project
3. Set up a Memorystore for Redis Instance
4. Import the data used by your application

## Install a Local Colab Runtime

Memorystore for Redis uses private, internal IP addresses for security. To access your Redis instance from the public internet, you'll need to set up a jump box and SSH into it from your Colab with port forwarding. Using a local Colab runtime makes this process easier.

Download and run the latest Colab runtime docker image on your local machine: `docker run --network=host us-docker.pkg.dev/colab-images/public/runtime`

Connect Colab to the local Colab runtime. Please refer to the [instructions](https://research.google.com/colaboratory/local-runtimes.html).

## Install Dependencies

To start, you'll need to install the dependencies for this demo app.

In [ ]:
%pip install --quiet langchain-google-memorystore-redis
# install additional depenedencies for the demo
%pip install --quiet langchain langchain-google-vertexai redis

## Authenticate to Google Cloud within Colab

Authentice the local Colab runtime to Google Cloud as well.

In [ ]:
!gcloud auth login
!gcloud auth application-default login

## Configure Your Project

In [12]:
# @markdown Please fill in the value below with your GCP project ID/preferred region and then run the cell.

# Please fill in these values.
project_id = "your_project_id"  # @param {type:"string"}
region = "us-central1"  # @param {type:"string"}

# Get a list of zones in the region and pick one
zone = get_ipython().getoutput(f'gcloud compute zones list --filter="region:({region})" --format="value(name)" | head -n 1')[0]

## Connect Your Google Cloud Project

To leverage Google Cloud from within Colab, you'll need to connect your Google Cloud Project to this notebook.

In [ ]:
# Quick input validations.
assert project_id, "⚠️ Please provide a Google Cloud project ID"

# Configure gcloud.
!gcloud config set project {project_id}
!gcloud auth application-default set-quota-project {project_id}

## Configure Your Google Cloud Project

Enable the APIs for Vertex AI within your project

In [ ]:
# Enable GCP services
!gcloud services enable aiplatform.googleapis.com redis.googleapis.com

## Set up Memorystore for Redis
You will need a Memorystore for Redis instance for the following stages of this notebook.

Running the below cell will verify the existence of the Memorystore for Redis instance and or create a new instance if one does not exist.
> ⏳ - Creating a Memorystore for Redis instance may take a few minutes.

In [ ]:
# @markdown Please fill in the both the Google Cloud region and name of your Memorystore for Redis instance. Once filled in, run the cell.

# Please fill in these values.
instance_name = "redis-langchain"  # @param {type:"string"}
instance_size_gb = 100  # @param {type:"string"}

# Quick input validations.
assert project_id, "⚠️ Please provide a Google Cloud project ID"
assert region, "⚠️ Please provide a Google Cloud region"
assert instance_name, "⚠️ Please provide the name of your instance"
assert instance_size_gb, "⚠️ Please provide the size of your instance in GB"


# Check if Memorystore for Redis instance exists in the provided region with the correct engine version
# TODO: Remove "beta" after GA
redis_version = !gcloud beta redis instances describe {instance_name}  --project {project_id} --region {region} --format="value(redisVersion)"
if redis_version == ["REDIS_7_2"]:
    print("Found an existing Memorystore for Redis Instance!")
else:
    print("Creating a new Memorystore for Redis instance...")
    !gcloud beta redis instances create --region {region} {instance_name} --size {instance_size_gb} --zone {zone} --tier BASIC --redis-version=redis_7_2

**Important Note**: Memorystore for Redis uses private, internal IP addresses for security. This means you'll need to create a Google Compute Engine (GCE) VM within the same VPC as your Redis instance to connect.

In [ ]:
# Please fill in these values.
vm_name = "redis-jump-box" #@param {type:"string"}

# Quick input validations.
assert project_id, "⚠️ Please provide a Google Cloud project ID"
assert region, "⚠️ Please provide a Google Cloud region"
assert vm_name, "⚠️ Please provide the name of your vm"

vm_exists = !gcloud compute instances describe {vm_name} --project={project_id} --zone={zone}

if "ERROR" in vm_exists[0]:
    print("Creating a new Redis jump box ...")
    !gcloud compute instances create {vm_name} \
        --project={project_id} \
        --zone={zone} \
        --machine-type=e2-medium \
        --network-interface=network-tier=PREMIUM,stack-type=IPV4_ONLY,subnet=default \
        --maintenance-policy=MIGRATE \
        --provisioning-model=STANDARD \
        --scopes=https://www.googleapis.com/auth/cloud-platform \
        --no-shielded-secure-boot \
        --shielded-vtpm \
        --shielded-integrity-monitoring \
        --labels=goog-ec-src=vm_add-gcloud \
        --reservation-affinity=any
else:
    print("Found an existing Redis jump box!")

To connect to your Redis instance, you'll need to establish an SSH tunnel using port forwarding through your Google Compute Engine (GCE) VM.

In [ ]:
redis_ip = !gcloud beta redis instances describe {instance_name} --region {region} --format="value(host)"
!gcloud compute ssh --project={project_id} --zone={region}-a {vm_name} -- -NL 6379:{redis_ip}:6379

## Import data to your Redis instance

Now that you have your Redis instance, you will need to import data! You will be using a [Netflix Dataset from Kaggle](https://www.kaggle.com/datasets/shivamb/netflix-shows). Here is what the data looks like:

| show_id | type    | title                | director         | cast                                                                                                                                                  | country       | date_added        | release_year | rating | duration  | listed_in                                    | description                                                                                                                                                                           |
|---------|---------|----------------------|------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------|---------------|-------------------|--------------|--------|-----------|----------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| s1      | Movie   | Dick Johnson Is Dead | Kirsten Johnson  |                                                                                                                                                        | United States | September 25, 2021 | 2020         | PG-13  | 90 min    | Documentaries                                | As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable.                              |
| s2      | TV Show | Blood & Water        |                  | Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba, Dillon Windvogel, Natasha Thahane, Arno Greeff, Xolile Tshabalala, Getmore Sithole, Cindy Mahlangu, Ryle De Morny, Greteli Fincham, Sello Maake Ka-Ncube, Odwa Gwanya, Mekaila Mathys, Sandi Schultz, Duane Williams, Shamilla Miller, Patrick Mofokeng | South Africa  | September 24, 2021 | 2021         | TV-MA  | 2 Seasons | International TV Shows, TV Dramas, TV Mysteries | After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth.                                   |
| s3      | TV Show | Ganglands            | Julien Leclercq  | Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabiha Akkari, Sofia Lesaffre, Salim Kechiouche, Noureddine Farihi, Geert Van Rampelberg, Bakary Diombera                                   |               | September 24, 2021 | 2021         | TV-MA  | 1 Season  | Crime TV Shows, International TV Shows, TV Action & Adventure | To protect his family from a powerful drug lord, skilled thief Mehdi and his expert team of robbers are pulled into a violent and deadly turf war.                                     |
| s4      | TV Show | Jailbirds New Orleans |                  |                                                                                                                                                        |               | September 24, 2021 | 2021         | TV-MA  | 1 Season  | Docuseries, Reality TV                        | Feuds, flirtations and toilet talk go down among the incarcerated women at the Orleans Justice Center in New Orleans on this gritty reality series.                                   |
| s5      | TV Show | Kota Factory         |                  | Mayur More, Jitendra Kumar, Ranjan Raj, Alam Khan, Ahsaas Channa, Revathi Pillai, Urvi Singh, Arun Kumar                                                 | India        | September 24, 2021 | 2021         | TV-MA  | 2 Seasons | International TV Shows, Romantic TV Shows, TV Comedies | In a city of coaching centers known to train India’s finest collegiate minds, an earnest but unexceptional student and his friends navigate campus life. |


Create a Redis client:

In [21]:
import redis

# Since using SSH, "localhost" (127.0.0.1) is the address for the Redis instance
client = redis.from_url(f"redis://127.0.0.1:6379")

Download the dataset:

In [22]:
from google.cloud import storage

# Initialize the Google Cloud Storage client
gcs_client = storage.Client()

# Download the CSV file
bucket_name = "cloud-samples-data"
source_blob_name = "langchain/netflix_titles_compute_embeddings.csv"
csv_file_path = "./netflix_titles_compute_embeddings.csv"
gcs_client.bucket(bucket_name).blob(source_blob_name).download_to_filename(
    csv_file_path
)

Load the data into Redis:

In [29]:
import csv
import redis

# Create a pipeline
pipeline = client.pipeline()

# Path to the CSV file
csv_file_path = "./netflix_titles_compute_embeddings.csv"

batch_size = 500

with open(csv_file_path, mode="r", encoding="utf-8") as file:
    reader = csv.DictReader(file)
    count = 0  # Initialize a counter to keep track of batch size

    for row in reader:
        redis_key = f"netflix:{row['show_id']}"
        redis_hash = {k: v for k, v in row.items() if k != "show_id"}

        # Add the HSET operation to the pipeline
        pipeline.hset(redis_key, mapping=redis_hash)

        count += 1  # Increment the counter for each operation added

        # Execute the pipeline every 100 operations
        if count % batch_size == 0:
            pipeline.execute()
            pipeline = client.pipeline()  # Reset the pipeline after execution

    # Execute any remaining commands in the pipeline
    pipeline.execute()

# Use case 1: Memorystore for Redis as a Document Loader

Now that you have data in your Redis instance, you're ready to use it as a document loader. This means you'll pull data from the Redis instance and load it into memory as documents, ready to be added to your vector store.

To use your Memorystore for Redis instance as a document loader, you'll utilize the MemorystoreDocumentLoader class.

The "content_fields" argument specifies which fields within your Redis HASH keys will become the "content" of your documents. Any other fields within those HASH keys will be treated as "metadata" attached to each document.

In [30]:
from langchain_google_memorystore_redis import MemorystoreDocumentLoader

content_fields = ["title", "director", "cast", "description"]
loader = MemorystoreDocumentLoader(
    client=client, key_prefix="netflix:", content_fields=content_fields
)

 Run the `lazy_load` function to pull documents from your Redis instance. Take a look at the first 5 documents below. Awesome, you've successfully used Memorystore for Redis as a document loader!

In [ ]:
iterator = loader.lazy_load()
for i in range(5):
    print(next(iterator))

# Use case 2: Memorystore for Redis as a Vector Store

Now, you'll learn how to put all of the documents you just loaded into a vector store backed by Memorystore for Redis. This will allow you to use vector search to answer user questions!

To create a vector store, you'll start by initializing it using the `init_index` function. Include all the fields you want for metadata, and specify a vector size (768) that matches the output of your embedding model (Vertex AI's textembedding-gecko in this case).

In [32]:
import re
from langchain_google_memorystore_redis import (
    DistanceStrategy,
    HNSWConfig,
    RedisVectorStore,
)

index_config = HNSWConfig(
    name="netflix_vs:", distance_strategy=DistanceStrategy.COSINE, vector_size=768
)

try:
    RedisVectorStore.init_index(client=client, index_config=index_config)
except redis.exceptions.ResponseError as e:
    if re.match(r".*already exists", str(e)):
        print("Index already exists, skipping creation.")
    else:
        raise

Initialize your embedding service. Here, you're using version 003 of Vertex AI's textembedding-gecko model. Specifying the model version is important for reproducibility and managing updates. Note that for each insertion, the embedding service will be called to compute the embeddings to store in the vector table. Pricing details can be found [here](https://cloud.google.com/vertex-ai/pricing).

In [33]:
from langchain_google_vertexai import VertexAIEmbeddings

embeddings_service = VertexAIEmbeddings(
    model_name="textembedding-gecko@003", project=project_id
)

Next, create a vector_store object linked to your Memorystore for Redis instance and then load your documents into the vector store.  **Important**: for each document, the embedding service will compute embeddings for storage – please refer to pricing details [here](https://cloud.google.com/vertex-ai/pricing).

In [34]:
vector_store = RedisVectorStore(
    client=client, index_name="netflix_vs:", embeddings=embeddings_service
)

You're ready to populate your vector store. Here's the code to start loading documents:
> ⏳ - Loading 8800+ documents might take a few seconds. Network speed will affect the time.

In [ ]:
docs_to_load = loader.load()

Now, you can add the loaded documents into the vector store backed by Memorystore for Redis.

> ⏳ - This process can take up to a minute. The vector store will call Vertex AI's embedding service to create embeddings for each document and store them in Redis.

In [ ]:
doc_ids = vector_store.add_documents(docs_to_load)
print(doc_ids[:5])

# Use case 3: Memorystore for Redis as Chat Memory

You'll now add chat history (or "memory") for more advanced LLM interactions.   This allows the LLM to retain context from past conversations, leading to better responses. Memorystore for Redis will be your memory storage.  To start,  let's initialize Memorystore for Redis for this purpose.

In [43]:
from langchain_google_memorystore_redis import MemorystoreChatMessageHistory

chat_history = MemorystoreChatMessageHistory(
    client=client,
    session_id="my_session",
)

To manage chat history, you'll typically need to alternate between adding user messages and AI responses to your memory storage. Here's a basic example of how you might implement this in code.

In [ ]:
chat_history.add_user_message("Hi!")
chat_history.add_ai_message("Hello there. I'm a model and am happy to help!")

chat_history.messages

# Conversational RAG Chain Backed by Memorystore for Redis

You've already explored Memorystore for Redis in several roles –  now it's time to integrate it into the ConversationalRetrievalChain!  You'll build a movie chatbot that leverages vector search for insightful answers.

Customize your LLM prompt with instructions that improve the bot's responses. For example, a reminder like 'Don't make things up' can increase accuracy in your movie chatbot.

In [45]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="""Use all the information from the context and the conversation history to answer new question. If you see the answer in previous conversation history or the context. \
Answer it with clarifying the source information. If you don't see it in the context or the chat history, just say you \
didn't find the answer in the given data. Don't make things up.

Previous conversation history from the questioner. "Human" was the user who's asking the new question. "Assistant" was you as the assistant:
```{chat_history}
```

Vector search result of the new question:
```{context}
```

New Question:
```{question}```

Answer:""",
    input_variables=["context", "question", "chat_history"],
)
condense_question_prompt_passthrough = PromptTemplate(
    template="""Repeat the following question:
{question}
""",
    input_variables=["question"],
)

Now, you'll use your vector store as a retriever with LangChain. This will allow you to directly "fetch" relevant documents based on your queries.

In [46]:
retriever = vector_store.as_retriever(
    search_type="mmr", search_kwargs={"k": 5, "lambda_mult": 0.8}
)

Now, initialize your LLM. For this example, you're using Vertex AI's "gemini-pro" model.

In [47]:
from langchain_google_vertexai import VertexAI

llm = VertexAI(model_name="gemini-pro", project=project_id)

Clear the chat history to ensure a clean conversational experience every time.

In [48]:
from langchain.memory import ConversationSummaryBufferMemory

chat_history.clear()
memory = ConversationSummaryBufferMemory(
    llm=llm,
    chat_memory=chat_history,
    output_key="answer",
    memory_key="chat_history",
    return_messages=True,
)

Now, create a ConversationalRetrievalChain. This will empower the LLM to incorporate chat history into its responses. You'll be able to ask follow-up questions without needing to provide full context every time.

In [49]:
from langchain.chains import ConversationalRetrievalChain

rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    verbose=False,
    memory=memory,
    condense_question_prompt=condense_question_prompt_passthrough,
    combine_docs_chain_kwargs={"prompt": prompt},
)

Start chatting with your movie chatbot!

In [ ]:
# Suppress all deprecation warnings
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

q = "What movie was Brad Pitt in?"
ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]
print(f"Question: {q}\nAnswer: {ans}\n")

q = "How about Jonny Depp?"
ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]
print(f"Question: {q}\nAnswer: {ans}\n")

q = "Are there movies about animals?"
ans = rag_chain({"question": q, "chat_history": chat_history})["answer"]
print(f"Question: {q}\nAnswer: {ans}\n")